In [1]:
import pandas as pd
import os

from pygridder import pygridder as pgrid
import pyproj
import pathlib

import multiprocessing.popen_spawn_posix
import dask
from dask.distributed import Client, progress

import skimage.morphology as skmorph
import datetime as dt
import numpy as np
import scipy.ndimage as ndimage

from glob import glob

In [34]:
import dclasses as dc
from scipy import stats

In [75]:
import gzip
import pygrib as pg

In [52]:
client = Client(processes=True)

In [90]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [4]:
# Global Variables & Pre-Processing for PP forecast
dx = 5 # delta x
selem = skmorph.disk(40 / dx) # morphology disk

ndfd_path = pathlib.Path('../scripts/impacts-data/pas-input-data/ndfd.npz').resolve()
with np.load(ndfd_path) as NPZ:
    lons = NPZ['lons']
    lats = NPZ['lats']
    
G = pgrid.Gridder(tx=lons, ty=lats, dx=dx/100)

dateparser = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') + dt.timedelta(hours=6)
data_path = pathlib.Path('../../../gen-assets/1950-2019_tors_CONUS.csv')

df = pd.read_csv(data_path, parse_dates=[['date','time']], date_parser=dateparser, index_col=0, keep_date_col=True)
df = df.reset_index()

all_di = {0: 0.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0}
sig_di = {0: 0.0, 1: 0.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0}
df['all_weight'] = df['mag'].map(all_di).fillna(df['mag'])
df['sig_weight'] = df['mag'].map(sig_di).fillna(df['mag'])

outlook_time = '1200'

# Create date array
my_hour = dt.datetime.strptime(outlook_time, '%H%M').hour
my_minute = dt.datetime.strptime(outlook_time, '%H%M').minute

bdt = dt.datetime(1950,1,1,my_hour,my_minute)
edt = dt.datetime(2019,12,31,my_hour,my_minute)

# Create list index of datetimes with a frequency of one per day
dts = pd.date_range(bdt, edt, freq='D')
bdts, edts = dts[:-1],dts[1:]

In [76]:
# PAS Global Variables and Pre-Processing
### Parse CLI Arguments ###

ndfd_area = 25
nsims = 10000
tornado_direction_distribution = stats.norm(50, 15)
coolseason = [1, 2, 3, 4, 11, 12]

impacts_data_root = pathlib.Path('../scripts/impacts-data/pas-input-data/')
outdir = pathlib.Path('./PAS-climo/',"output").resolve()
outdir.mkdir(exist_ok=True)

- loop through one tor
- add days to a das delayed if they include tornadoes
- once delayed arr is of length 12, run a compute...
    - Create tor and sigtor prob arrays
    - run PAS on these prob arrays and save psv.gz files

### Current Implementation of Dask to parallelize via grouping PP creation in chunks

In [62]:
## Baseline loop

%%time
for bdt, edt in zip(bdts[:100],edts[:100]):
    _df = df[(df['date_time'] >= bdt) & (df['date_time'] < edt) & (df['all_weight'] == 1.0)]
    if _df.empty:
        continue
    else:
        processPP(_df,bdt)

CPU times: user 8.3 s, sys: 118 ms, total: 8.42 s
Wall time: 7.38 s


In [88]:
%%time

da_list = []

for bdt, edt in zip(bdts[:100],edts[:100]):
    _df = df[(df['date_time'] >= bdt) & (df['date_time'] < edt) & (df['all_weight'] == 1.0)]
    if _df.empty:
        continue
    else:
        da = dask.delayed(processPP)(_df,bdt)
        da_list.append(da)
        
        if len(da_list) == 12:
            das = dask.compute(da_list)
            
            da_list = []
    

CPU times: user 3.31 s, sys: 2.17 s, total: 5.47 s
Wall time: 8.23 s


In [86]:
def runPAS(all_fcst, sig_fcst, date_in_name):
    
    outfile = outdir.joinpath(f"{date_in_name.strftime('%Y%m%d%H%M%S')}_ts.psv.gz")
    
    torn = all_fcst*100
    continuous_torn = dc.make_continuous(torn)
    sigtorn = sig_fcst*100
    
    sigtorn[sigtorn > 0] = 1
    if (torn.max() >= 30) and (sigtorn.max() > 0):
        sigtorn[torn >= 15] += 1
    sigtorn_1d = sigtorn.ravel()
    usesig = True if (date_in_name.month in coolseason) or (sigtorn.max() > 0) else False
    
    tornado_dists = dc.TornadoDistributions()
    counts = np.zeros((5, nsims), dtype=int)
    counts[0, :] = (tornado_dists.f02.rvs(nsims) * ndfd_area * (torn == 2).sum()).astype(int)
    counts[1, :] = (tornado_dists.f05.rvs(nsims) * ndfd_area * (torn == 5).sum()).astype(int)
    counts[2, :] = (tornado_dists.f10.rvs(nsims) * ndfd_area * (torn == 10).sum()).astype(int)
    counts[3, :] = (tornado_dists.f15.rvs(nsims) * ndfd_area * (torn == 15).sum()).astype(int)
    counts[4, :] = (tornado_dists.f30.rvs(nsims) * ndfd_area * (torn >= 30).sum()).astype(int)

In [87]:
def processPP(_df,date_in_name):
    
    lon1 = _df.slon.values
    lat1 = _df.slat.values
    lon2 = _df.elon.values
    lat2 = _df.elat.values

    # Find/remove/replace missing data
    keep = ~np.logical_or(lon1 == 0, lat1 == 0)
    lon1 = lon1[keep]
    lat1 = lat1[keep]
    lon2 = lon2[keep]
    lat2 = lat2[keep]
    lon2[lon2 == 0] = lon1[lon2 == 0]
    lat2[lat2 == 0] = lat1[lat2 == 0]

    # Grid tornadoes
    tornlines = G.grid_lines(sxs=lon1, sys=lat1, exs=lon2, eys=lat2)
    all_mags = _df['all_weight']
    sig_mags = _df['sig_weight']
    all_fcst = G.make_empty_grid(dtype='float')
    sig_fcst = G.make_empty_grid(dtype='float')
    
    for tornline, all_mag, sig_mag in zip(tornlines, all_mags, sig_mags):
        all_fcst[tornline] = all_mag
        sig_fcst[tornline] = sig_mag
    
    # Make practically perfect forecast
    all_fcst = skmorph.binary_dilation(all_fcst, selem).astype(float)
    sig_fcst = skmorph.binary_dilation(sig_fcst, selem).astype(float)

    #print(np.max(fcst))
    all_fcst = ndimage.gaussian_filter(all_fcst, 120/dx)
    sig_fcst = ndimage.gaussian_filter(sig_fcst, 120/dx)
    
    # Degrade continuous probs into SPC prob
    # Uncomment if regular probs
    all_fcst[all_fcst < 0.02] = 0
    all_fcst[np.logical_and(all_fcst < 0.05, all_fcst >= 0.02)] = 0.02
    all_fcst[np.logical_and(all_fcst < 0.10, all_fcst >= 0.05)] = 0.05
    all_fcst[np.logical_and(all_fcst < 0.15, all_fcst >= 0.10)] = 0.10
    all_fcst[np.logical_and(all_fcst < 0.30, all_fcst >= 0.15)] = 0.15
    all_fcst[np.logical_and(all_fcst < 0.45, all_fcst >= 0.30)] = 0.30
    all_fcst[all_fcst >= 0.45] = 0.45

    # Uncomment if sigtor probs
    sig_fcst[sig_fcst < 0.10] = 0
    sig_fcst[sig_fcst >= 0.10] = 0.10
    
    # Have this function call PAS function here
    runPAS(all_fcst, sig_fcst, date_in_name)
    
    #return all_fcst, sig_fcst

### Trying different ways to chunk data for dask parallelizing

In [72]:
%%time

finished = dask.compute(anom(bdts,edts))

CPU times: user 3.41 s, sys: 2.91 s, total: 6.32 s
Wall time: 7.52 s


In [71]:
def anom(bdts,edts):
    
    results = []
    
    for bdt, edt in zip(bdts[:100],edts[:100]):
        _df = df[(df['date_time'] >= bdt) & (df['date_time'] < edt) & (df['all_weight'] == 1.0)]
        if _df.empty:
            continue
        else:
            result = processPP_da(_df,bdt)
            
        results.append(result)
            
    return results

In [60]:
@dask.delayed
def processPP_da(_df,date_in_name):
    
    dtime = date_in_name.strftime('%Y%m%d%H%M%S')
    
    lon1 = _df.slon.values
    lat1 = _df.slat.values
    lon2 = _df.elon.values
    lat2 = _df.elat.values

    # Find/remove/replace missing data
    keep = ~np.logical_or(lon1 == 0, lat1 == 0)
    lon1 = lon1[keep]
    lat1 = lat1[keep]
    lon2 = lon2[keep]
    lat2 = lat2[keep]
    lon2[lon2 == 0] = lon1[lon2 == 0]
    lat2[lat2 == 0] = lat1[lat2 == 0]

    # Grid tornadoes
    tornlines = G.grid_lines(sxs=lon1, sys=lat1, exs=lon2, eys=lat2)
    all_mags = _df['all_weight']
    sig_mags = _df['sig_weight']
    all_fcst = G.make_empty_grid(dtype='float')
    sig_fcst = G.make_empty_grid(dtype='float')
    
    for tornline, all_mag, sig_mag in zip(tornlines, all_mags, sig_mags):
        all_fcst[tornline] = all_mag
        sig_fcst[tornline] = sig_mag
    
    # Make practically perfect forecast
    all_fcst = skmorph.binary_dilation(all_fcst, selem).astype(float)
    sig_fcst = skmorph.binary_dilation(sig_fcst, selem).astype(float)

    #print(np.max(fcst))
    all_fcst = ndimage.gaussian_filter(all_fcst, 120/dx)
    sig_fcst = ndimage.gaussian_filter(sig_fcst, 120/dx)
    
    # Degrade continuous probs into SPC prob
    # Uncomment if regular probs
    all_fcst[all_fcst < 0.02] = 0
    all_fcst[np.logical_and(all_fcst < 0.05, all_fcst >= 0.02)] = 0.02
    all_fcst[np.logical_and(all_fcst < 0.10, all_fcst >= 0.05)] = 0.05
    all_fcst[np.logical_and(all_fcst < 0.15, all_fcst >= 0.10)] = 0.10
    all_fcst[np.logical_and(all_fcst < 0.30, all_fcst >= 0.15)] = 0.15
    all_fcst[np.logical_and(all_fcst < 0.45, all_fcst >= 0.30)] = 0.30
    all_fcst[all_fcst >= 0.45] = 0.45

    # Uncomment if sigtor probs
    sig_fcst[sig_fcst < 0.10] = 0
    sig_fcst[sig_fcst >= 0.10] = 0.10
    
    # Have this function call PAS function here
    
    return [all_fcst, sig_fcst]